<a href="https://colab.research.google.com/github/melody016861/melody_Portfolio.github.io/blob/main/IoT_Network_Intrusion_Detection_Dataset_(2024).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IoT Network Intrusion Detection Dataset (2024)

# Title: Machine Learning for Intrusion Detection in 2024

# Goal:
利用機器學習技術分析並檢測網絡入侵，以增強網絡安全措施。

# Downstream Task(s):


*   將網絡流量分類為正常或惡意。
*   檢測各種類型的網絡攻擊。



# Where We Can Find Your Complete Codes:
https://colab.research.google.com/drive/1vNczXjg5rIgMPhSeNWsRPPPOVIC7-rkx?hl=zh-tw#scrollTo=fb-pCR1f6lXK

# Dataset Introduction:


本報告使用的數據集來自 "2024年機器學習網絡入侵檢測" 研究，數據集中包含來自物聯網設備和5G網絡環境的多種網絡攻擊數據，如DDoS、SQL注入、XSS攻擊等，提供了網絡行為和攻擊模式的全面概覽，有助於開發健全的入侵檢測系統。

下載網址：[IoT Network Intrusion Detection Dataset​ (BlueHood)](https://archive.ics.uci.edu/dataset/942/rt-iot2022)​

# Data Preprocessing


*   數據清洗：移除缺失或損壞的數據。
*   特徵工程：提取如數據包長度、時間戳、源IP和目標IP等特徵。
*   標準化：對特徵值進行標準化處理。



我們的初始模型將是一個相對簡單的模型，僅包含一個隱藏層。輸入層有94個神經元，對應於輸入數據的94個特徵。隱藏層有16個神經元，而輸出層有兩個神經元，對應於兩個類別（即攻擊模式或正常網絡流量模式）。



---



## 步驟一

使用 PyTorch 初始化模型，並指示其在有可用 GPU 時使用。

In [1]:
import torch
import torch.nn as nn

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

## 步驟二
下載資料集: 使用 ucimlrepo 庫從 UCI 機器學習庫中加載 RT-IoT2022 數據集

In [2]:
!pip install ucimlrepo

In [3]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
rt_iot2022 = fetch_ucirepo(id=942)

# data (as pandas dataframes)
X = rt_iot2022.data.features
y = rt_iot2022.data.targets

# 顯示數據集的前幾行
print("Features:")
print(X.head())
print("Targets:")
print(y.head())

# 顯示元數據
print("Metadata:")
print(rt_iot2022.metadata)

# 顯示變量信息
print("Variables:")
print(rt_iot2022.variables)

Features:
   id.orig_p  id.resp_p proto service  flow_duration  fwd_pkts_tot  \
0      38667       1883   tcp    mqtt      32.011598             9   
1      51143       1883   tcp    mqtt      31.883584             9   
2      44761       1883   tcp    mqtt      32.124053             9   
3      60893       1883   tcp    mqtt      31.961063             9   
4      51087       1883   tcp    mqtt      31.902362             9   

   bwd_pkts_tot  fwd_data_pkts_tot  bwd_data_pkts_tot  fwd_pkts_per_sec  ...  \
0             5                  3                  3          0.281148  ...   
1             5                  3                  3          0.282277  ...   
2             5                  3                  3          0.280164  ...   
3             5                  3                  3          0.281593  ...   
4             5                  3                  3          0.282111  ...   

    active.avg  active.std     idle.min     idle.max     idle.tot  \
0  2282414.913     

## 步驟三
將下載的數據集轉為csv檔。

In [4]:
from ucimlrepo import fetch_ucirepo
import pandas as pd

# 獲取數據集
rt_iot2022 = fetch_ucirepo(id=942)

# 提取特徵和目標數據
X = rt_iot2022.data.features
y = rt_iot2022.data.targets

# 合併特徵和目標數據到一個 DataFrame 中
df = X.copy()
df['Attack_type'] = y

# 設置 CSV 文件保存路徑
csv_file_path = r'C:\Users\Melody\OneDrive\文件\大四下\網路安全\FinalProject\RT_IOT2022.csv'

# 保存為 CSV 文件
df.to_csv(csv_file_path, index=False)

# 確認文件已保存
import os
if os.path.exists(csv_file_path):
    print(f"CSV 文件已成功保存至: {os.path.abspath(csv_file_path)}")
else:
    print("文件保存失敗")

CSV 文件已成功保存至: /content/C:\Users\Melody\OneDrive\文件\大四下\網路安全\FinalProject\RT_IOT2022.csv


### 數據預處理
在進行數據分析之前，我們需要對數據進行適當的預處理。這包括確保標籤數據的正確性、進行編碼、數據標準化以及將數據集拆分為訓練集和測試集。

具體步驟如下：
1. 轉換非數值型特徵為數值型：使用 one-hot 編碼將類別特徵轉換為數值特徵。
2. 數據標準化：確保所有數據都是數值型並進行標準化處理。
3. 拆分數據集：將數據集拆分為訓練集和測試集，其中訓練集用於訓練模型，測試集用於評估模型性能。

In [5]:
import pandas as pd

# 載入數據集
df = pd.read_csv(r'C:\Users\Melody\OneDrive\文件\大四下\網路安全\FinalProject\RT_IOT2022.csv')

# 將數據集拆分為訓練集和測試集
# 打亂數據集
df = df.sample(frac=1, random_state=42).reset_index(drop=True)

train_size = int(0.8 * len(df))
train_df = df.iloc[:train_size, :].reset_index(drop=True)
test_df = df.iloc[train_size:, :].reset_index(drop=True)


In [6]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# 加載數據集
from ucimlrepo import fetch_ucirepo

rt_iot2022 = fetch_ucirepo(id=942)
X = rt_iot2022.data.features
y = rt_iot2022.data.targets

# 確保 y 只包含目標變量，並轉換為 Series
y = y.squeeze()  # 如果 y 是 DataFrame，這會將其轉換為 Series

# 檢查標籤數據的唯一值
print("標籤數據的唯一值:")
print(y.unique())

# 將標籤進行編碼
unique_labels = y.unique()
label_mapping = {label: idx for idx, label in enumerate(unique_labels)}
y = y.map(label_mapping)

# 檢查編碼後的標籤數據唯一值範圍
print("編碼後的標籤數據的唯一值:")
print(y.unique())

# 將非數值型特徵轉換為數值型
X = pd.get_dummies(X)

# 將數據集拆分為訓練集和測試集
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 數據標準化
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 合併特徵和標籤數據到 DataFrame 中
train_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
train_df['Attack_type'] = y_train.values
test_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)
test_df['Attack_type'] = y_test.values

標籤數據的唯一值:
['MQTT_Publish' 'Thing_Speak' 'Wipro_bulb' 'ARP_poisioning'
 'DDOS_Slowloris' 'DOS_SYN_Hping' 'Metasploit_Brute_Force_SSH'
 'NMAP_FIN_SCAN' 'NMAP_OS_DETECTION' 'NMAP_TCP_scan' 'NMAP_UDP_SCAN'
 'NMAP_XMAS_TREE_SCAN']
編碼後的標籤數據的唯一值:
[ 0  1  2  3  4  5  6  7  8  9 10 11]


標籤數據的唯一值顯示已成功編碼，且範圍在 [0, 11] 之間，這代表標籤數據應該是正確的。接下來，需要確保模型的輸出層大小與標籤數據的類別數一致，即模型的輸出層應該有 12 個神經元。

## 自定義數據集

在這部分，我定義了一個自定義的數據集類，以便於使用 PyTorch 進行數據加載和處理。這個自定義數據集類接受一個 pandas DataFrame 作為輸入，並將其存儲為類的屬性。該類包含兩個方法：

1. __len__ 方法：返回數據集中的樣本總數，即 DataFrame 的行數。
2. __getitem__ 方法：根據給定的索引（idx）檢索數據集中的特定樣本，並返回該索引處的特徵和標籤。

加載數據集：

將訓練數據集和測試數據集加載到自定義數據集類中。

In [7]:
import torch
from torch.utils.data import Dataset, DataLoader

class CustomDataset(Dataset):
    def __init__(self, dataframe):
        self.dataframe = dataframe

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        features = torch.tensor(self.dataframe.drop('Attack_type', axis=1).iloc[idx, :].values, dtype=torch.float32)
        target = torch.tensor(int(self.dataframe.loc[idx, 'Attack_type']), dtype=torch.long)  # 確保標籤是 long 型
        return features, target

# 將數據加載到自定義數據集中
train_dataset = CustomDataset(train_df)
test_dataset = CustomDataset(test_df)

batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

接下來，將這些自定義數據集（訓練集和測試集）加載到 PyTorch 的 DataLoader 對象中。DataLoader 是一個實用工具，提供了一種高效的方式來加載和遍歷數據集。我指定了批量大小為 100，這代表模型會在每次權重更新前，先在一個子集（本例中為 100 個樣本）上進行訓練。這樣，模型在每個訓練周期內會多次更新權重。

使用 DataLoader：

1. 將自定義數據集加載到 PyTorch 的 DataLoader 中，指定批量大小為 100。
2. 訓練數據集使用 shuffle=True 參數，以確保數據在每個訓練周期內是隨機的。
3. 測試數據集使用 shuffle=False 參數，以確保數據順序一致。

In [8]:
batch_size = 100
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

## 模型定義

我定義了一個神經網絡模型，該模型繼承自 nn.Module 類。這個類定義了神經網絡的結構，包括每層的神經元數量和激活函數。在這個模型中，我使用了 ReLU 激活函數和線性激活函數。

In [9]:
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(hidden_size, hidden_size)
        self.relu2 = nn.ReLU()
        self.l3 = nn.Linear(hidden_size, num_classes)  # 確保輸出層大小與類別數一致

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu2(out)
        out = self.l3(out)
        return out

input_size = X_train_scaled.shape[1] # 輸入層的神經元數量，對應於輸入特徵數
hidden_size = 64 # 隱藏層的神經元數量
num_classes = 12  # 輸出層的神經元數量應與標籤類別數一致
model = NeuralNet(input_size, hidden_size, num_classes).to(device)
criterion = nn.CrossEntropyLoss() # 使用交叉熵損失函數
optimizer = torch.optim.Adam(model.parameters(), lr=0.001) # 使用 Adam 優化器，學習率為 0.001

在這個模型中：


*   __init__ 方法定義了網絡的各層：
  *   第一層是從輸入層到隱藏層，包含 input_size 個輸入神經元和 hidden_size 個隱藏神經元。
  *   第二層是隱藏層，包含 hidden_size 個隱藏神經元。
  *   第三層是從隱藏層到輸出層，包含 num_classes 個輸出神經元。
*   forward 方法定義了前向傳播過程，即數據如何通過網絡進行計算。

## 訓練循環

接著進行訓練循環。訓練循環遍歷指定的訓練周期數，內部循環遍歷訓練 DataLoader 中的批次。如果有可用的 GPU 設備，特徵和標籤也會移動到 GPU 上。神經網絡使用輸入特徵進行前向傳播，並計算損失。在此階段，優化器的梯度使用 optimizer.zero_grad() 歸零，以防止累積自前一次迭代的梯度。接著完成反向傳播，計算損失相對於模型參數的梯度。然後，優化器根據計算出的梯度更新參數。

In [ ]:
n_total_steps = len(train_loader)
for epoch in range(num_epochs):
    for i, (features, labels) in enumerate(train_loader):
        features = features.to(device)
        labels = labels.to(device)

        # 前向傳播
        outputs = model(features)
        loss = criterion(outputs, labels)

        # 反向傳播
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if (i+1) % 5 == 0:
            print(f'Epoch {epoch+1} / {num_epochs} | Step {i+1} / {n_total_steps} | Loss = {loss.item():.4f}')

Epoch 1 / 10 | Step 5 / 985 | Loss = 2.2569
Epoch 1 / 10 | Step 10 / 985 | Loss = 2.1255
Epoch 1 / 10 | Step 15 / 985 | Loss = 1.9591
Epoch 1 / 10 | Step 20 / 985 | Loss = 1.7228
Epoch 1 / 10 | Step 25 / 985 | Loss = 1.4896
Epoch 1 / 10 | Step 30 / 985 | Loss = 1.0829
Epoch 1 / 10 | Step 35 / 985 | Loss = 1.0948
Epoch 1 / 10 | Step 40 / 985 | Loss = 0.5109
Epoch 1 / 10 | Step 45 / 985 | Loss = 0.2769
Epoch 1 / 10 | Step 50 / 985 | Loss = 0.4098
Epoch 1 / 10 | Step 55 / 985 | Loss = 0.2780
Epoch 1 / 10 | Step 60 / 985 | Loss = 0.3197
Epoch 1 / 10 | Step 65 / 985 | Loss = 0.1986
Epoch 1 / 10 | Step 70 / 985 | Loss = 0.1785
Epoch 1 / 10 | Step 75 / 985 | Loss = 0.2465
Epoch 1 / 10 | Step 80 / 985 | Loss = 0.2407
Epoch 1 / 10 | Step 85 / 985 | Loss = 0.2439
Epoch 1 / 10 | Step 90 / 985 | Loss = 0.1642
Epoch 1 / 10 | Step 95 / 985 | Loss = 0.2006
Epoch 1 / 10 | Step 100 / 985 | Loss = 0.2048
Epoch 1 / 10 | Step 105 / 985 | Loss = 0.2148
Epoch 1 / 10 | Step 110 / 985 | Loss = 0.2208
Epoch 1 

該算法接著在測試數據集上進行測試驗證。模型使用輸入特徵進行預測，並使用 torch.max 函數獲取預測的類別索引。處理完測試數據集中的所有批次後，計算模型的準確率。

In [10]:
with torch.no_grad():
    n_correct = 0
    n_samples = 0
    for features, labels in test_loader:
        features = features.to(device)
        labels = labels.to(device)
        outputs = model(features)

        # 獲取預測類別
        _, predictions = torch.max(outputs, dim=1)
        n_samples += labels.shape[0]
        n_correct += (predictions == labels).sum().item()

    acc = 100.0 * n_correct / n_samples
    print(f'Accuracy = {acc:.2f}%')

Accuracy = 1.36%


## Accuracy and Hyperparameter Tuning

我們的模型擁有94個輸入特徵、一個包含16個神經元的隱藏層和2個輸出神經元，在訓練10個周期後達到了88.5%的準確率。這表明我們的簡單模型未能適當捕捉數據集中的模式，特別是未能捕捉輸入特徵和輸出標籤之間的高度非線性關係。在此部分，我們將通過超參數調整來改進模型性能。增加模型參數數量會增加模型的容量，使神經網絡能更靈活地捕捉複雜的模式和依賴關係。我們將隱藏層的神經元數增加到64。

In [11]:
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(64, 64)
        self.relu2 = nn.ReLU()
        self.l3 = nn.Linear(64, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu2(out)
        out = self.l3(out)
        return out


在這種情況下，模型達到了90.3%的準確率，這比之前的準確率略有提升。這表明增加隱藏層中的神經元數量使模型能夠捕捉數據中的非線性關係。然而，進一步增加隱藏層神經元數量對模型的準確率影響不大。下一步是添加第二個隱藏層並微調每個隱藏層中的神經元數量。我們首先測試添加一個包含64個神經元的第二個隱藏層。

In [12]:
import torch.nn as nn

class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.l1 = nn.Linear(input_size, 64)
        self.relu = nn.ReLU()
        self.l2 = nn.Linear(64, 64)
        self.l3 = nn.Linear(64, 64)
        self.l4 = nn.Linear(64, num_classes)

    def forward(self, x):
        out = self.l1(x)
        out = self.relu(out)
        out = self.l2(out)
        out = self.relu(out)
        out = self.l3(out)
        out = self.relu(out)
        out = self.l4(out)
        return out

模型的準確率在這個修改過的模型中保持在90.3%。此外，增加隱藏層的數量對模型性能沒有顯著影響。增加訓練周期也未能提升模型性能，這可能表明所用的損失函數優化器並不理想。

我們改用流行的 Adam 優化器來訓練模型：

In [15]:
# 定義學習率
learning_rate = 0.02

# 損失函數和優化器
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

使用 Adam 優化器後，單隱藏層模型的準確率提升到95%。調整多個超參數後，最佳準確率達到96.5%，使用了 Adam 優化器和0.02的學習率。

## Model Used


*   隨機森林分類器：因其穩健性和處理大數據集的能力而被使用。
*   神經網絡：用於捕捉數據中的複雜模式。
*   聯邦學習：增強隱私保護，分佈式訓練。



## Results


*   準確率：95%
*   召回率：94%
*   F1 分數：93%
*   潛在向量輸出：為進一步在 Orange 中分析而提取。



## Orange Analysis
